In [1]:
import pandas as pd
import numpy as np
from pqd_dataset import PQDDataset
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.utils.data import DataLoader
from pqd_trainer import train_model, evaluate_model
from models.conv_mlp import GRU
import random
import os

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, h_n = self.gru(x)
        out = self.fc(h_n[-1])  # Use the last hidden state
        return out

# Define input size, hidden size, and number of classes
input_size = 5  # 5 time series
hidden_size = 64
num_classes = 3

# Create the model
model = GRUModel(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Print the model architecture
print(model)

# Example of how to train the model (assuming you have X_train and y_train)
# X_train should have shape (num_samples, 32, 5)
# y_train should have shape (num_samples,)

# Convert data to PyTorch tensors
X_train = torch.randn(100, 32, 5)  # Example data
y_train = torch.randint(0, 3, (100,))  # Example labels

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Example of how to evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_train).sum().item() / y_train.size(0)
    print(f'Accuracy: {accuracy:.4f}')

GRUModel(
  (gru): GRU(5, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)
Epoch [1/10], Loss: 1.1078
Epoch [2/10], Loss: 1.1035
Epoch [3/10], Loss: 1.0994
Epoch [4/10], Loss: 1.0954
Epoch [5/10], Loss: 1.0916
Epoch [6/10], Loss: 1.0878
Epoch [7/10], Loss: 1.0842
Epoch [8/10], Loss: 1.0807
Epoch [9/10], Loss: 1.0772
Epoch [10/10], Loss: 1.0739
Accuracy: 0.4700


In [3]:
torch.save(model, "gru.pt")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Dropout, Input
from tensorflow.keras.models import Model

def create_gru_model(num_sensors=5, num_states=3, hidden_dim=60, num_layers=1, dropout=0.4):
    inputs = Input(shape=(None, num_sensors))

    gru_out, _ = GRU(hidden_dim, return_sequences=False, return_state=True)(inputs)

    linear1 = Dense(hidden_dim, activation='relu')(gru_out)
    linear1 = Dropout(dropout)(linear1)

    linear2 = Dense(num_states)(linear1)

    model = Model(inputs=inputs, outputs=linear2)
    return model

# Create the model
model = create_gru_model()

# Print the model summary
model.summary()

# Convert the model to TensorFlow Lite format with CPU optimizations
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply default optimizations for CPU
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model_cpu_optimized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-09-01 13:15:28.213226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-01 13:15:28.963634: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-01 13:15:29.747456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-01 13:15:29.747829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ [(None, 60), (None,    │        12,060 │
│                                 │ 60)]                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           183 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,903 (62.12 KB)

 Trainable params: 15,903 (62.12 KB)

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: /tmp/tmphvwjriyv/assets


INFO:tensorflow:Assets written to: /tmp/tmphvwjriyv/assets


In [2]:
def seed_everything(seed: int = 42):
    """
    This function is used to maintain repeatability
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
seed = 42
seed_everything(seed)

In [4]:
window_size = 32
stride = 100
target_mode = True
epochs=10
batch_size=1
lr = 0.0005

In [5]:
train_df = pd.read_csv('train.csv')
train_dataset = PQDDataset(
                            df=train_df,
                            window_size=window_size,
                            stride=stride,
                            target_mode=target_mode
                            )

Creating window slices:   0%|          | 0/66 [00:00<?, ?it/s]

In [6]:
weights = compute_class_weight("balanced", classes=np.unique(train_dataset.df.target), y=train_dataset.df.target)
weights = torch.FloatTensor(weights)

In [7]:
test_df = pd.read_csv('test.csv')
test_dataset = PQDDataset(
                            df=test_df,
                            window_size=window_size,
                            stride=stride,
                            target_mode=target_mode
                            )

Creating window slices:   0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
model = GRU()

In [9]:
sum(p.numel() for p in model.parameters())

15903

In [10]:
train_model(
            model=model,
            dataset=train_dataset,
            run_seed=seed,
            eval_dataset=test_dataset,
            epochs=epochs,
            batch_size=batch_size,
            lr=lr,
            weights=None)

Epoch ...:   0%|          | 0/10 [00:00<?, ?it/s]

Step ...:   0%|          | 0/662 [00:00<?, ?it/s]

Step ...:   0%|          | 0/180 [00:00<?, ?it/s]

[0.93385214 0.82474227 0.        ]
epoch 0  0.5861981360396861


Step ...:   0%|          | 0/662 [00:00<?, ?it/s]

...

Step ...:   0%|          | 0/662 [00:00<?, ?it/s]

Step ...:   0%|          | 0/180 [00:00<?, ?it/s]

[0.95967742 0.86792453 0.        ]
epoch 5  0.6092006492189085


Step ...:   0%|          | 0/662 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
torch.save(model, "gru.pt")

### evaluate_model(model, test_dataset)

In [2]:
random.randint(0, 256)

22

In [3]:
random.randint(0, 256)

39

In [4]:
random.randint(0, 256)

108

In [5]:
random.randint(0, 256)

182

In [6]:
random.randint(0, 256)

171

In [12]:
v = torch.zeros(1, 3)

In [13]:
v

tensor([[0., 0., 0.]])

In [15]:
v[:,2] = 1

In [16]:
v

tensor([[0., 0., 1.]])